[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/contextual_chunk_headers.ipynb)

# 上下文块头 (CCH)

## 概述

上下文块头 (CCH) 是一种创建包含更高级别上下文（例如文档级或节级上下文）的块头的方法，并在嵌入块之前将这些块头附加到块上。这使得嵌入能够更准确、更完整地表示文本的内容和含义。在我们的测试中，此功能显著提高了检索质量。除了提高检索到正确信息的比率外，CCH 还降低了搜索结果中出现不相关结果的比率。这降低了 LLM 在下游聊天和生成应用程序中误解一段文本的比率。

## 动机

开发人员在使用 RAG 时面临的许多问题都归结于此：单个块通常不包含足够的上下文，无法被检索系统或 LLM 正确使用。这导致无法回答问题，更令人担忧的是，还会产生幻觉。

此问题的示例
- 块通常通过隐式引用和代词来指代其主题。这导致它们在应该被检索时没有被检索到，或者没有被 LLM 正确理解。
- 单个块通常只有在整个节或文档的上下文中才有意义，单独阅读时可能会产生误导。

## 关键组件

#### 上下文块头
这里的想法是通过在块前面添加一个块头来为块添加更高级别的上下文。这个块头可以像文档标题一样简单，也可以使用文档标题、简洁的文档摘要以及节和子节标题的完整层次结构的组合。

## 方法详情

#### 上下文生成
在下面的演示中，我们使用 LLM 为文档生成一个描述性标题。这是通过一个简单的提示完成的，您将文档文本的截断版本传递给 LLM，并要求 LLM 为文档生成一个描述性标题。如果您已经有足够描述性的文档标题，则可以直接使用它们。我们发现，文档标题是包含在块头中最简单、最重要的更高级别上下文。

您可以包含在块头中的其他类型的上下文：
- 简洁的文档摘要
- 节/子节标题
    - 这有助于检索系统处理对文档中较大部分或主题的查询。

#### 嵌入带有块头的块
您为每个块嵌入的文本只是块头和块文本的串联。如果您在检索期间使用重新排序器，您需要确保在那里也使用相同的串联。

#### 将块头添加到搜索结果中
在向 LLM 呈现搜索结果时包含块头也是有益的，因为它为 LLM 提供了更多上下文，并使其不太可能误解块的含义。

![Your Technique Name](../images/contextual_chunk_headers.svg)

## 设置

您需要一个 Cohere API 密钥和一个 OpenAI API 密钥才能使用此笔记本。

# 包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [ ]:
# Install required packages
!pip install langchain openai python-dotenv tiktoken

In [3]:
import cohere
import tiktoken
from typing import List
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load environment variables from a .env file
load_dotenv()
os.environ["CO_API_KEY"] = os.getenv('CO_API_KEY') # Cohere API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key

## 加载文档并将其拆分为块
在此演示中，我们将使用基本的 LangChain RecursiveCharacterTextSplitter，但您可以将 CCH 与更复杂的分块方法相结合，以获得更好的性能。

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/nike_2023_annual_report.txt https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/nike_2023_annual_report.txt


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_into_chunks(text: str, chunk_size: int = 800) -> list[str]:
    """
    Split a given text into chunks of specified size using RecursiveCharacterTextSplitter.

    Args:
        text (str): The input text to be split into chunks.
        chunk_size (int, optional): The maximum size of each chunk. Defaults to 800.

    Returns:
        list[str]: A list of text chunks.

    Example:
        >>> text = "This is a sample text to be split into chunks."
        >>> chunks = split_into_chunks(text, chunk_size=10)
        >>> print(chunks)
        ['This is a', 'sample', 'text to', 'be split', 'into', 'chunks.']
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=0,
        length_function=len
    )
    documents = text_splitter.create_documents([text])
    return [document.page_content for document in documents]

# File path for the input document
FILE_PATH = "data/nike_2023_annual_report.txt"

# Read the document and split it into chunks
with open(FILE_PATH, "r") as file:
    document_text = file.read()

chunks = split_into_chunks(document_text, chunk_size=800)

## 生成用于块头的描述性文档标题

In [5]:
# Constants
DOCUMENT_TITLE_PROMPT = """
INSTRUCTIONS
What is the title of the following document?

Your response MUST be the title of the document, and nothing else. DO NOT respond with anything else.

{document_title_guidance}

{truncation_message}

DOCUMENT
{document_text}
""".strip()

TRUNCATION_MESSAGE = """
Also note that the document text provided below is just the first ~{num_words} words of the document. That should be plenty for this task. Your response should still pertain to the entire document, not just the text provided below.
""".strip()

MAX_CONTENT_TOKENS = 4000
MODEL_NAME = "gpt-4o-mini"
TOKEN_ENCODER = tiktoken.encoding_for_model('gpt-3.5-turbo')

def make_llm_call(chat_messages: list[dict]) -> str:
    """
    Make an API call to the OpenAI language model.

    Args:
        chat_messages (list[dict]): A list of message dictionaries for the chat completion.

    Returns:
        str: The generated response from the language model.
    """
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=chat_messages,
        max_tokens=MAX_CONTENT_TOKENS,
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()

def truncate_content(content: str, max_tokens: int) -> tuple[str, int]:
    """
    Truncate the content to a specified maximum number of tokens.

    Args:
        content (str): The input text to be truncated.
        max_tokens (int): The maximum number of tokens to keep.

    Returns:
        tuple[str, int]: A tuple containing the truncated content and the number of tokens.
    """
    tokens = TOKEN_ENCODER.encode(content, disallowed_special=())
    truncated_tokens = tokens[:max_tokens]
    return TOKEN_ENCODER.decode(truncated_tokens), min(len(tokens), max_tokens)

def get_document_title(document_text: str, document_title_guidance: str = "") -> str:
    """
    Extract the title of a document using a language model.

    Args:
        document_text (str): The text of the document.
        document_title_guidance (str, optional): Additional guidance for title extraction. Defaults to "".

    Returns:
        str: The extracted document title.
    """
    # Truncate the content if it's too long
    document_text, num_tokens = truncate_content(document_text, MAX_CONTENT_TOKENS)
    truncation_message = TRUNCATION_MESSAGE.format(num_words=3000) if num_tokens >= MAX_CONTENT_TOKENS else ""

    # Prepare the prompt for title extraction
    prompt = DOCUMENT_TITLE_PROMPT.format(
        document_title_guidance=document_title_guidance,
        document_text=document_text,
        truncation_message=truncation_message
    )
    chat_messages = [{"role": "user", "content": prompt}]
    
    return make_llm_call(chat_messages)

# Example usage
if __name__ == "__main__":
    # Assuming document_text is defined elsewhere
    document_title = get_document_title(document_text)
    print(f"Document Title: {document_title}")

NIKE, INC. ANNUAL REPORT ON FORM 10-K


## Add chunk header and measure impact
Let's look at a specific example to demonstrate the impact of adding a chunk header. We'll use the Cohere reranker to measure relevance to a query with and without a chunk header.

In [13]:
def rerank_documents(query: str, chunks: List[str]) -> List[float]:
    """
    Use Cohere Rerank API to rerank the search results.

    Args:
        query (str): The search query.
        chunks (List[str]): List of document chunks to be reranked.

    Returns:
        List[float]: List of similarity scores for each chunk, in the original order.
    """
    MODEL = "rerank-english-v3.0"
    client = cohere.Client(api_key=os.environ["CO_API_KEY"])

    reranked_results = client.rerank(model=MODEL, query=query, documents=chunks)
    results = reranked_results.results
    reranked_indices = [result.index for result in results]
    reranked_similarity_scores = [result.relevance_score for result in results]
    
    # Convert back to order of original documents
    similarity_scores = [0] * len(chunks)
    for i, index in enumerate(reranked_indices):
        similarity_scores[index] = reranked_similarity_scores[i]

    return similarity_scores

def compare_chunk_similarities(chunk_index: int, chunks: List[str], document_title: str, query: str) -> None:
    """
    Compare similarity scores for a chunk with and without a contextual header.

    Args:
        chunk_index (int): Index of the chunk to inspect.
        chunks (List[str]): List of all document chunks.
        document_title (str): Title of the document.
        query (str): The search query to use for comparison.

    Prints:
        Chunk header, chunk text, query, and similarity scores with and without the header.
    """
    chunk_text = chunks[chunk_index]
    chunk_wo_header = chunk_text
    chunk_w_header = f"Document Title: {document_title}\n\n{chunk_text}"

    similarity_scores = rerank_documents(query, [chunk_wo_header, chunk_w_header])

    print(f"\nChunk header:\nDocument Title: {document_title}")
    print(f"\nChunk text:\n{chunk_text}")
    print(f"\nQuery: {query}")
    print(f"\nSimilarity without contextual chunk header: {similarity_scores[0]:.4f}")
    print(f"Similarity with contextual chunk header: {similarity_scores[1]:.4f}")

# Notebook cell for execution
# Assuming chunks and document_title are defined in previous cells
CHUNK_INDEX_TO_INSPECT = 86
QUERY = "Nike climate change impact"

compare_chunk_similarities(CHUNK_INDEX_TO_INSPECT, chunks, document_title, QUERY)


Chunk header:
Document Title: NIKE, INC. ANNUAL REPORT ON FORM 10-K

Chunk text:
Given the broad and global scope of our operations, we are particularly vulnerable to the physical risks of climate change, such 
as shifts in weather patterns. Extreme weather conditions in the areas in which our retail stores, suppliers, manufacturers, 
customers, distribution centers, offices, headquarters and vendors are located could adversely affect our operating results and 
financial condition. Moreover, natural disasters such as earthquakes, hurricanes, wildfires, tsunamis, floods or droughts, whether 
occurring in the United States or abroad, and their related consequences and effects, including energy shortages and public 
health issues, have in the past temporarily disrupted, and could in the future disrupt, our operations, the operations of our

Query: Nike climate change impact

Similarity w/o contextual chunk header: 0.10576342
Similarity with contextual chunk header: 0.92206234


This chunk is clearly about the impact of climate change on some organization, but it doesn't explicitly say "Nike" in it. So the relevance to the query "Nike climate change impact" in only about 0.1. By simply adding the document title to the chunk that similarity goes up to 0.92.

# Eval results

#### KITE

We evaluated CCH on an end-to-end RAG benchmark we created, called KITE (Knowledge-Intensive Task Evaluation).

KITE currently consists of 4 datasets and a total of 50 questions.
- **AI Papers** - ~100 academic papers about AI and RAG, downloaded from arXiv in PDF form.
- **BVP Cloud 10-Ks** - 10-Ks for all companies in the Bessemer Cloud Index (~70 of them), in PDF form.
- **Sourcegraph Company Handbook** - ~800 markdown files, with their original directory structure, downloaded from Sourcegraph's publicly accessible company handbook GitHub [page](https://github.com/sourcegraph/handbook/tree/main/content).
- **Supreme Court Opinions** - All Supreme Court opinions from Term Year 2022 (delivered from January '23 to June '23), downloaded from the official Supreme Court [website](https://www.supremecourt.gov/opinions/slipopinion/22) in PDF form.

Ground truth answers are included with each sample. Most samples also include grading rubrics. Grading is done on a scale of 0-10 for each question, with a strong LLM doing the grading.

We compare performance with and without CCH. For the CCH config we use document title and document summary. All other parameters remain the same between the two configurations. We use the Cohere 3 reranker, and we use GPT-4o for response generation.

|                         | No-CCH   | CCH          |
|-------------------------|----------|--------------|
| AI Papers               | 4.5      | 4.7          |
| BVP Cloud               | 2.6      | 6.3          |
| Sourcegraph             | 5.7      | 5.8          |
| Supreme Court Opinions  | 6.1      | 7.4          |
| **Average**             | 4.72     | 6.04         |

We can see that CCH leads to an improvement in performance on each of the four datasets. Some datasets see a large improvement while others see a small improvement. The overall average score increases from 4.72 -> 6.04, a 27.9% increase.

#### FinanceBench

We've also evaluated CCH on FinanceBench, where it contributed to a score of 83%, compared to a baseline score of 19%. For that benchmark, we tested CCH and relevant segment extraction (RSE) jointly, so we can't say exactly how much CCH contributed to that result. But the combination of CCH and RSE clearly leads to substantial accuracy improvements on FinanceBench.